In [146]:
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### url : https://www.kaggle.com/datasets/jackywang529/michelin-restaurants

In [147]:
data_1 = pd.read_csv('/kaggle/input/michelin-restaurants/one-star-michelin-restaurants.csv')
data_2 = pd.read_csv('/kaggle/input/michelin-restaurants/three-stars-michelin-restaurants.csv')
data_3 = pd.read_csv('/kaggle/input/michelin-restaurants/two-stars-michelin-restaurants.csv')

data = pd.concat([data_1,data_2,data_3],axis=0).sample(frac=1.0).reset_index(drop=True)

In [148]:
data

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Kashiwaya,2019,22.280643,114.156740,Hong Kong,Hong Kong,NaN,Japanese,$$$$,https://guide.michelin.com/hk/en/hong-kong-reg...
1,Le Bernardin,2019,40.761770,-73.982230,New York,New York City,10019,Seafood,$$$$,https://guide.michelin.com/us/en/new-york-stat...
2,Kitchen W8,2019,51.499180,-0.196920,Kensington,United Kingdom,W8 6AH,Modern cuisine,NaN,https://guide.michelin.com/gb/en/greater-londo...
3,Kosaka,2019,40.738316,-74.001370,New York,New York City,10011,Japanese,$$$$,https://guide.michelin.com/us/en/new-york-stat...
4,Sun Tung Lok,2019,22.300990,114.172210,Hong Kong,Hong Kong,NaN,Cantonese,$$,https://guide.michelin.com/hk/en/hong-kong-reg...
...,...,...,...,...,...,...,...,...,...,...
690,L'Enclume,2019,54.201885,-2.953807,Cartmel,United Kingdom,LA11 6PZ,Creative,NaN,https://guide.michelin.com/gb/en/cumbria/cartm...
691,Faro,2019,40.707584,-73.923035,New York,New York City,11237,American,$$,https://guide.michelin.com/us/en/new-york-stat...
692,Star Inn at Harome,2019,54.231010,-1.010230,Harome,United Kingdom,YO62 5JE,Modern British,NaN,https://guide.michelin.com/gb/en/north-yorkshi...
693,Roganic,2019,51.518314,-0.152087,Marylebone,United Kingdom,W1U 3DB,Creative British,NaN,https://guide.michelin.com/gb/en/greater-londo...


In [149]:
data = data.drop(['url','name','zipCode'],axis=1)

In [150]:
data.isnull().sum()

year           0
latitude       0
longitude      0
city           2
region         0
cuisine        0
price        176
dtype: int64

In [151]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{'city': array(['Hong Kong', 'New York', 'Kensington', 'Singapore', 'Salzburg',
        'Gaillimh/Galway', 'Soho', 'San Francisco', 'Birkenhead', 'Seoul',
        'Skåne-Tranås', 'Rio de Janeiro - 22271', 'Washington, D.C.',
        'Macau', 'Helsingfors / Helsinki', 'Taipei', 'Aughton',
        'Los Angeles', 'Järpen', 'Bermondsey', 'Menai Bridge/Porthaethwy',
        'Mayfair', 'Newbury', 'Wien', 'South San Francisco', 'Marylebone',
        'Birmingham', 'København', 'Hyde Park', 'City of London', 'Aarhus',
        'Bloomsbury', 'Llandrillo', 'Chicago', 'Stavanger',
        'Saint Helier/Saint-Hélier', 'Stockholm', 'Göteborg', 'Chagford',
        'Chelsea', 'Kew', 'Rio de Janeiro - 22441', 'Bangkok', 'Oxford',
        'Nottingham', 'Stratford-upon-Avon', 'Llanddewi Skirrid',
        'Trondheim', 'Winteringham', 'São Paulo - 04538',
        'Upper Hambleton', 'South Dalton', 'Bath', 'Baslow',
        "Saint James's", 'Rio de Janeiro - 22021', nan,
        'São Paulo - 05415', 'São Pau

In [152]:
data['price'].mode()## i think we should be do something else
data['price'] = data['price'].replace(np.nan,'$$$$')
#data = data.dropna(axis=0)

In [153]:
data = data.dropna(axis=0)

In [154]:
data['city'] = data['city'].apply(lambda x:re.sub(r' - \d+$/gm','',x))

In [155]:
nominal_fets = ['cuisine','region','city']
nominal_prefix = ['CU_','REG_','CI_']

for col , prefix in zip(nominal_fets,nominal_prefix):
    dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_32/1529391747.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
/tmp/ipykernel_32/1529391747.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
/tmp/ipykernel_32/1529391747.py:5: Deprecation

In [156]:
ordinal_fets = [ '$','$$','$$$','$$$$','$$$$$']
data['price'] = data['price'].apply(lambda x: ordinal_fets.index(x))

In [157]:
y = data['price']
x = data.drop('price',axis=1)

In [158]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [159]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.75,random_state=13)

In [163]:
log = LogisticRegression(C= 0.017)
log.fit(x_train,y_train)
log.score(x_test,y_test)

0.632183908045977

In [172]:
svm = SVC(C=3.121)
svm.fit(x_train,y_train)
svm.score(x_test,y_test)


0.632183908045977